# 1. Perprocess
## 1.1 parse XML
use BeautifulSoup to parse XML file, output with pandas dataframe "sentence, E#A, sentiment"

In [1]:
def parseXML(path):
    with open(path) as xmldata:
        soup = BeautifulSoup(xmldata, "xml")
    # Create empty lists to store the extracted data
    sentence_ids = []
    texts = []
    categories = []
    polarities = []
    # Loop through the 'sentence' elements and extract the necessary information
    for sentence in soup.find_all('sentence'):
        opinions = sentence.find('Opinions')
        if opinions is not None:
            s_categories = []
            s_polarities = []
            for opinion in opinions.find_all('Opinion'):
                s_categories.append(opinion['category'])
                s_polarities.append(opinion['polarity'])
            categories.append(s_categories)
            polarities.append(s_polarities)
        else:
            continue
        sentence_ids.append(sentence['id'])
        texts.append(sentence.find('text').text)

    # Create a pandas dataframe from the extracted data
    df = pd.DataFrame({'Sentence ID': sentence_ids,
                       'Text': texts,
                       'Categories': categories,
                       'Polarities': polarities})
    print(f'{path.split("/")[-1]} has been parsed. The number of sentences with opinions is '
          f"{len(soup.find_all('Opinions'))}({len(soup.find_all('sentence'))}).")
    
    return df

In [2]:
# # Data analysis
# import matplotlib.pyplot as plt

# # Split categories and create new rows
# df_copy = df.copy()
# split_categories = df_copy['Category'].str.split(',')
# df_copy = df_copy.assign(Category=split_categories).explode('Category')

# # Get the unique categories and their frequencies
# category_counts = df_copy['Category'].value_counts()
# #top_categories = category_counts.head(50)

# # Plot the frequencies in descending order
# category_counts.plot(kind='barh')
# plt.gca().invert_yaxis()
# plt.title('Category Frequencies')
# plt.xlabel('Frequency')
# plt.ylabel('Category')
# plt.show()

## 1.2 tokenisation, (stopwords,punc removal), lemmatization

In [3]:
def tokenisation(data):
    tokenised_data = [word_tokenize(sentence) for sentence in data]
    return tokenised_data


def remove_stopwords(data):
    stopwords = stopwords.words('english')
    filtered_words = [[word for word in sentence if word.lower() not in stopwords] for sentence in data]
    return filter_words
#print(stopwords.words('english'))
def remove_punctuation(data):
    text = [[word for word in sentence if re.sub(r'[^\w\s]+', '', word).isalnum()] for sentence in data]
    return text

In [4]:
##lemmatization
from nltk import stem
def lemmatization(data):
    wnl = stem.WordNetLemmatizer()
    lematized = [[wnl.lemmatize(word) for word in sentence] for sentence in tokenised_data]
    return lematized

In [5]:
#data = df['Text'].values.tolist()
#data = tokenisation(data)
#data = remove_stopwords(data)
#data = remove_punctuation(data)
#data = lemmatization(data)
#data = [[word.lower() for word in sentence] for sentence in data]
#print(data)

# 2. Model - Max Entropy Classifier with threshold t
## 2.1 Features

In [6]:
# from nltk.tag import pos_tag
# pos_data = [pos_tag(sentence,tagset='universal') for sentence in data]
# print(pos_data)

In [7]:
import re
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, classification_report

In [8]:
def feature_bow_around_verb(sentences):
    sentence_list = []
    verb_bow_list = []

    for sentence in sentences:
        verb_bow = []
        pos_tags = nltk.pos_tag(sentence)

        verbs = [word for word, pos in pos_tags if pos.startswith('VB')]

        if len(verbs) > 0:
            verb = verbs[0]
            verb_index = sentence.index(verb)

            for i in range(max(0, verb_index - 5), verb_index):
                if pos_tags[i][1].startswith('JJ') or pos_tags[i][1].startswith('RB') or pos_tags[i][1].startswith('NN'):
                    verb_bow.append(sentence[i])

            for i in range(verb_index + 1, min(verb_index + 6, len(sentence))):
                if pos_tags[i][1].startswith('JJ') or pos_tags[i][1].startswith('RB') or pos_tags[i][1].startswith('NN'):
                    verb_bow.append(sentence[i])

        sentence_list.append(sentence)
        verb_bow_list.append(verb_bow)

    df = pd.DataFrame({'Sentence': sentence_list, 'BoW around Verb': verb_bow_list})
    return df

In [9]:
def feature_bow_at_end_of_sentence(sentences):
    sentence_list = []
    end_of_sentence_bow_list = []

    for sentence in sentences:
        end_of_sentence_bow = []
        pos_tags = nltk.pos_tag(sentence)

        for i in range(len(pos_tags)-1, max(len(pos_tags)-6, -1), -1):
            if pos_tags[i][1].startswith('JJ') or pos_tags[i][1].startswith('RB'):
                end_of_sentence_bow.append(sentence[i])

        sentence_list.append(sentence)
        end_of_sentence_bow_list.append(end_of_sentence_bow)

    df = pd.DataFrame({'Sentence': sentence_list, 'BoW at End of Sentence': end_of_sentence_bow_list})
    return df

In [10]:
def custom_tokenizer(text):
    # Tokenize the text using NLTK's word_tokenize function
    tokens = word_tokenize(text)
    # Remove punctuation
    tokens = [word for word in tokens if re.sub(r'[^\w\s]+', '', word).isalnum()]
    # Remove stopwords
    stop_words = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop_words]
    # Convert tokens to lowercase
    tokens = [token.lower() for token in tokens]   
    return tokens

def feature_unigram(df, df_t):    
    # Create a CountVectorizer instance to compute unigram counts
    count_vectorizer = CountVectorizer(tokenizer=custom_tokenizer,max_features=1000)
    # Fit and transform the sentences to obtain the unigram count features
    count_matrix = count_vectorizer.fit_transform(df['text'])
    count_matrix_t = count_vectorizer.transform(df_t['text'])
    # Convert the count matrix to a dataframe with appropriate column names
    count_df = pd.DataFrame(count_matrix.toarray(), columns=count_vectorizer.get_feature_names())
    count_df_t = pd.DataFrame(count_matrix_t.toarray(), columns=count_vectorizer.get_feature_names())
    return count_df, count_df_t

def feature_Tfidf(df, df_t):
    # Create a TfidfVectorizer instance to compute TF-IDF features
    tfidf_vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer)
    # Fit and transform the sentences to obtain the TF-IDF features
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])
    tfidf_matrix_t = tfidf_vectorizer.transform(df_t['text'])
    # Convert the TF-IDF matrix to a dataframe with appropriate column names
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names())
    tfidf_df_t = pd.DataFrame(tfidf_matrix_t.toarray(), columns=tfidf_vectorizer.get_feature_names())
    return tfidf_df, tfidf_df_t

In [11]:
def catagory_extraction_LogisticRegressionClf():
    #data preprocessing
    Laptops_Train_p1 = parseXML("./data/Laptops_Train_p1.xml")
    Laptops_Test_p1_gold = parseXML("./data/Laptops_Test_p1_gold.xml")

    train = pd.DataFrame({'text': Laptops_Train_p1['Text'],
                          'label': Laptops_Train_p1['Categories']})
    test = pd.DataFrame({'text': Laptops_Test_p1_gold['Text'],
                         'label': Laptops_Test_p1_gold['Categories']})

    # Concatenate the original dataframe with the unigram count dataframe and the TF-IDF dataframe
    unigram_train, unigram_test = feature_unigram(train, test)
    tfidf_train, tfidf_test = feature_Tfidf(train, test)
    x_train = pd.concat([unigram_train, tfidf_train], axis=1)
    x_test = pd.concat([unigram_test, tfidf_test], axis=1)
    # x_train = unigram_train
    # x_test = unigram_test

    # Convert the labels into binary arrays
    mlb = MultiLabelBinarizer()
    y = mlb.fit_transform(train['label'])
    y_test = mlb.transform(test['label'])
    # Create a dataframe for the binary label arrays
    y_train = pd.DataFrame(y, columns=mlb.classes_)
    y_test = pd.DataFrame(y_test, columns=mlb.classes_)
    
    # Create an instance of LogisticRegression
    clf = LogisticRegression(max_iter=1000)
    y_pred = np.zeros(y_test.shape)
    # Iterate through each label and train a separate binary classifier
    for i in range(y_train.shape[1]):
        label = mlb.classes_[i]
        print("Training classifier for label:", label)
        # Fit the model on the training data for the current label
        clf.fit(x_train, y_train.iloc[:, i])
        # Predict the probabilities of the current label for the testing data
        y_pred_prob = clf.predict_proba(x_test)[:, 1] # Use probabilities of positive class (1)

        # Set the threshold for category assignment
        threshold = 0.2

        # Generate predicted labels based on threshold
        y_pred_labels = np.where(y_pred_prob >= threshold, 1, 0)

        # Add predicted labels to the corresponding column in the binary label array
        y_pred[:, i] = y_pred_labels.tolist()

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

    # Generate classification report
    report = classification_report(y_test, y_pred, zero_division=1)
    print("Classification Report:\n", report)

In [12]:
def dataframe_expansion(df):
    original_labels = df['label'].tolist()
    original_polarities = df['polarity'].tolist()
    
    # ==advanced indexing==
    reps = [len(val) for val in df['label']]
    df = df.loc[np.repeat(df.index.values, reps)]
    
    df['label'] = [item for sublist in original_labels for item in sublist]
    df['polarity'] = [item for sublist in original_polarities for item in sublist]
    
    return df

def onehotting(x_train, x_test):
    # Convert 'label' column in x_train and x_test to one-hot encoding
    x_train_onehot = pd.get_dummies(x_train['label'], prefix='class')
    x_test_onehot = pd.get_dummies(x_test['label'], prefix='class')

    # Get the columns that are missing in x_test_onehot
    missing_cols = set(x_train_onehot.columns) - set(x_test_onehot.columns)
    for col in missing_cols:
        x_test_onehot[col] = 0

    # Reorder the columns in x_test_onehot to match the column names in x_train_onehot
    x_test_onehot = x_test_onehot[x_train_onehot.columns]
    
    # Concatenate x_train and x_test with one-hot encoded columns
    x_train = pd.concat([x_train, x_train_onehot], axis=1)
    x_test = pd.concat([x_test, x_test_onehot], axis=1)

    return x_train, x_test

In [13]:
def sentiment_analysis_feature_selection(features='unigram'):
    """
    parameter:
        
        features: String
        which consists of customised features "unigram,Tfidf,..." splitted by "," 
    
    returns:
        x_train, x_test, y_train, y_test = DataFrame
        only includes the data needed for training and test.
    """
    Laptops_Train_p1 = parseXML("./data/Laptops_Train_p1.xml")
    Laptops_Test_p1_gold = parseXML("./data/Laptops_Test_p1_gold.xml")
    train = pd.DataFrame({'text': Laptops_Train_p1['Text'],
                          'label': Laptops_Train_p1['Categories'],
                          'polarity': Laptops_Train_p1['Polarities']})
    test = pd.DataFrame({'text': Laptops_Test_p1_gold['Text'],
                         'label': Laptops_Test_p1_gold['Categories'],
                         'polarity': Laptops_Test_p1_gold['Polarities']})
    # Customised
    # Concatenate the original dataframe with the features selected
    # Ignore upper/lower case, leading or trailing whitespaces
    features_set = {'tfidf','unigram'}
    customised_features_set = set(feature.strip().lower() for feature in features.split(","))
    if not customised_features_set.issubset(features_set):
        raise ValueError("Please input with right features.")

    features_list = []
    # Appending tuples at first
    if 'tfidf' in customised_features_set:
        features_list.append(feature_unigram(train, test))
    if 'unigram' in customised_features_set:
        features_list.append(feature_Tfidf(train, test))
    # Unpacking
    x_train_features = []
    x_test_features = []
    for x_train_feature, x_test_feature in features_list:
        x_train_features.append(x_train_feature)
        x_test_features.append(x_test_feature)
    # convert to DataFrame
    x_train = pd.concat(x_train_features, axis=1)
    x_test = pd.concat(x_test_features, axis=1)
    
    # Default
    # numerical representation of polarity
    xy_train = dataframe_expansion(pd.concat([train, x_train], axis=1))
    xy_test = dataframe_expansion(pd.concat([test, x_test], axis=1))
    y_dict = {'positive': 1, 'negative': -1, 'neutral': 0}
    y_train = xy_train['polarity'].map(y_dict)
    y_test = xy_test['polarity'].map(y_dict)
    # label one-hotting
    x_train, x_test = onehotting(xy_train.drop(['text', 'polarity'], axis=1), xy_test.drop(['text', 'polarity'], axis=1))
    # Drop the original 'label' column from x_train and x_test
    x_train.drop('label', axis=1, inplace=True)
    x_test.drop('label', axis=1, inplace=True)
    
    return x_train, x_test, y_train, y_test

In [14]:
def sentiment_anaylysis_LogisticRegressionClf(features='unigram'):
    """
        parameter:
        
        features: String
        which consists of customised features "unigram,Tfidf,..." splitted by "," 
        and will be used by the training of classifier
    """
    # Feature selection
    x_train, x_test, y_train, y_test = sentiment_analysis_feature_selection(features=features)
        
    # Create an instance of LogisticRegression
    clf = LogisticRegression(max_iter=1000)
    clf.fit(x_train, y_train)

    # Make predictions on training data
    y_pred = clf.predict(x_test)

    # Generate classification report for training data
    classification_report_train = classification_report(y_test, y_pred)
    print("Classification Report (Training Data):\n", classification_report_train)

In [17]:
sentiment_anaylysis_LogisticRegressionClf(features='unigram')

Laptops_Train_p1.xml has been parsed. The number of sentences with opinions is 2039(2500).
Laptops_Test_p1_gold.xml has been parsed. The number of sentences with opinions is 808(808).
Classification Report (Training Data):
               precision    recall  f1-score   support

          -1       0.66      0.68      0.67       274
           0       0.33      0.07      0.11        46
           1       0.79      0.84      0.81       481

    accuracy                           0.74       801
   macro avg       0.60      0.53      0.53       801
weighted avg       0.72      0.74      0.72       801

